<a href="https://colab.research.google.com/github/sagittariusk2/fake-news-detection/blob/main/RR_version_of_fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.1 MB/s eta 0:00:00


In [3]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import re
import csv
import urllib.parse
from transformers import pipeline
import requests
import time
import json
import sys
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
csv.field_size_limit(sys.maxsize)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


9223372036854775807

In [ ]:
### Feature Extraction
    ## Complexity Features
        # 01. special_character <-
        # 02. upper_case <-
        # 03. lower_case <-
        # 04. short_sentences <-
        # 05. long_sentences <-
        # 06. question_marks <-
        # 07. exclamation_marks <-
    ## Stylometric Features
        # 08. adjective_count <-
        # 09. adverb_count <-
    ## 
        # 10. hyperlinks <-
        # 11. hyperlinks_diversity <-
        # 12. phising_score <-
        # 13. sensational_score <-
        # 14. sponsorship_score <-
        # 15. headline_content_symmetry_score <-

In [5]:
# 01. special_character

specialCharecters = "`~!@#$%^&*()_-+={}[]:;'<>,.?/\\|\""

def special_character_fe(content, headline=""):
  ans = 0
  for i in headline:
    if i in specialCharecters:
      ans += 1
  for i in content:
    if i in specialCharecters:
      ans += 1
  return ans

In [6]:
# 02. upper_case
def upper_case_fe(content, headline=""):
  ans = 0
  for i in headline:
    if i<='Z' and i>='A':
      ans += 1
  for i in content:
    if i<='Z' and i>='A':
      ans += 1
  return ans

In [7]:
# 03. lower_case
def lower_case_fe(content, headline=""):
  ans = 0
  for i in headline:
    if i<='z' and i>='a':
      ans += 1
  for i in content:
    if i<='z' and i>='a':
      ans += 1
  return ans

In [8]:
# 04. short_sentences
# 05. long_sentences

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead 
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

def length_sentences_fe(content, headline=""):
  headline_sentences = split_into_sentences(headline)
  content_sentences = split_into_sentences(content)
  short_sentence_cnt = 0
  long_sentence_cnt = 0
  for i in headline_sentences:
    if len(i)<=15:
      short_sentence_cnt += 1
    else:
      long_sentence_cnt += 1
  for i in content_sentences:
    if len(i)<=15:
      short_sentence_cnt += 1
    else:
      long_sentence_cnt += 1
  return [short_sentence_cnt, long_sentence_cnt]

In [9]:
# 06. Question Marks

def question_marks_fe(content, headline=""):
  ans = 0
  for i in headline:
    if i=='?':
      ans += 1
  for i in content:
    if i=='?':
      ans += 1
  return ans

In [10]:
# 07. Exclamation Marks

def exclamation_marks_fe(content, headline=""):
  ans = 0
  for i in headline:
    if i=='!':
      ans += 1
  for i in content:
    if i=='!':
      ans += 1
  return ans

In [32]:
# 08. adjective_count

def adjective_count_fe(content, headline=""):
  JJ_count = 0
  content_tokenized = sent_tokenize(content)
  for i in content_tokenized:
    content_token = word_tokenize(i)
    content_token = [w for w in content_token if not w in stop_words]
    content_tagged = pos_tag(content_token)
    for i in content_tagged:
      if i[1]=='JJ':
        JJ_count += 1

  headline_tokenized = sent_tokenize(headline)
  for i in headline_tokenized:
    headline_token = word_tokenize(i)
    headline_token = [w for w in headline_token if not w in stop_words]
    headline_tagged = pos_tag(headline_token)
    for i in headline_tokenized:
      if i[1]=='JJ':
        JJ_count += 1

  return JJ_count

3

In [ ]:
# 09. adverb_count

def adverb_count_fe(content, headline=""):
  RB_count = 0
  content_tokenized = sent_tokenize(content)
  for i in content_tokenized:
    content_token = word_tokenize(i)
    content_token = [w for w in content_token if not w in stop_words]
    content_tagged = pos_tag(content_token)
    for i in content_tagged:
      if i[1]=='RB':
        RB_count += 1

  headline_tokenized = sent_tokenize(headline)
  for i in headline_tokenized:
    headline_token = word_tokenize(i)
    headline_token = [w for w in headline_token if not w in stop_words]
    headline_tagged = pos_tag(headline_token)
    for i in headline_tokenized:
      if i[1]=='RB':
        RB_count += 1

  return RB_count

In [14]:
# 10. Hyperlinks
# 11. Hyperlinks Diversity
# 12. Phishing URL Score

phising_url_mpp = dict()

def phishing_url_score_fe(phising_url):
  API_key ='6874343ff58ee2ea10dae6ec40978e9647b03a1dbb72870295bcb540fd41738f'
  url = 'https://www.virustotal.com/vtapi/v2/url/report'
  res = []
  if phising_url in phising_url_mpp.keys():
    if phising_url_mpp[phising_url]!=900:
      return phising_url_mpp[phising_url]
  parameters = {'apikey': API_key, 'resource': url}
  try:
    time.sleep(16)
    response= requests.get(url=url, params=parameters,timeout=5)
    if response.status_code==200:
      json_response= json.loads(response.text)
      if json_response['response_code']==1:
        phising_url_mpp[phising_url]=json_response['positives']
      else:
        phising_url_mpp[phising_url]=90
    else:
      phising_url_mpp[phising_url]=90
  except:
    phising_url_mpp[phising_url]=900
  return phising_url_mpp[phising_url]

def hyperLinks_fe(content, headline=""):
  headline_urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', headline)
  content_urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', content)
  
  mpp = dict()

  for i in headline_urls:
    domain = urllib.parse.urlparse(i).netloc
    if domain in mpp.keys():
      mpp[domain] += 1
    else:
      mpp[domain] = 1
  
  for i in content_urls:
    domain = urllib.parse.urlparse(i).netloc
    if domain in mpp.keys():
      mpp[domain] += 1
    else:
      mpp[domain] = 1
  cnt = 0
  ans = 0
  phising_score = 0
  for i in mpp:
    cnt += mpp[i]
    ans += (mpp[i] * mpp[i])
    phising_score += phishing_url_score_fe(i)*mpp[i]
  return [cnt, ans, phising_score]


In [15]:
# 13. Sensational Score

def sentimental_score_fe(content, headline=""):
  sentiment_pipeline = pipeline("sentiment-analysis")
  return (sentiment_pipeline(headline+". "+content)[0]['score'])

In [ ]:
# 14. sponsorship_score

sponsorship_words = ['adherent', 'advocacy', 'advocate', 'aegis', 'aid', 'alms', 'angel', 'atonement', 'auspices', 'backer', 'backing', 'bene-faction', 'benefactor', 'beneficence', 'bid', 'charity', 'contribution', 'egis', 'endeavor', 'essay', 'expiation', 'feeler', 'gift', 'godparent', 'grubstaker', 'guarantor', 'guardianship', 'hit', 'mainstay', 'oblation', 'offer', 'offered', 'offering', 'overture', 'pass', 'patron', 'patron', 'patronage', 'pitch', 'present', 'presentation', 'promoter', 'proposition', 'propoundment', 'protectorship', 'rendition', 'sacrifice', 'sponsor', 'submission', 'subscription', 'support', 'supporter', 'surety', 'sustainer', 'tender', 'tutelage', 'underwriter']
def sponsorship_score_fe(content, headline=""):
  headline_token = word_tokenize(headline)
  content_token = word_tokenize(content)
  text_stem = []
  ans = 0
  lemmatizer = WordNetLemmatizer()
  for i in headline_token:
    text_stem.append(lemmatizer.lemmatize(i))
  for i in content_token:
    text_stem.append(lemmatizer.lemmatize(i))
  for i in text_stem:
    if i in sponsorship_words:
      ans += 1
  return ans

In [40]:
# 15. headline_content_symmetry_score

def preprocess_text(text):
  # Tokenize text
  tokens = word_tokenize(text)

  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

  # Lemmatize words
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(word.lower()) for word in filtered_tokens]

  # Return preprocessed text
  return ' '.join(lemmatized_tokens)

def headline_content_symmetry_score_fe(content, headline=""):
  if headline=="":
    return 1.0

  preprocessed_text1 = preprocess_text(content)
  preprocessed_text2 = preprocess_text(headline)

  vectorizer = CountVectorizer().fit_transform([preprocessed_text1, preprocessed_text2])
  similarity = cosine_similarity(vectorizer)[0][1]
  return similarity

In [52]:
def add_feature(news):
  news['question_marks'] = question_marks_fe(news['headline'], news['content'])
  news['exclamation_marks'] = exclamation_marks_fe(news['headline'], news['content'])
  hyperLinks = hyperLinks_fe(news['headline'], news['content'])
  news['hyperlinks'] = hyperLinks[0]
  news['hyperlinks_diversity'] = hyperLinks[1]
  news['phising_score'] = hyperLinks[2]
  # news['sensational_score'] = sentimental_fe(news['headline'], news['content'])
  return news

In [50]:
with open('phising_url.csv', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for i in reader:
    phising_url_mpp[i['url']]=int(i['score'])

In [ ]:
# Open files and extract feature

trained_dataset = []
id = 0

with open('news.csv', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for i in reader:
    label = 0
    if i['label']=='REAL':
      label = 1
    news = {'id':id, 'headline':i['title'], 'content':i['text'], 'label':label}
    news = add_feature(news)
    id += 1
    trained_dataset.append(news)

with open('train.csv', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for i in reader:
    news = {'id':id, 'headline':i['title'], 'content':i['text'], 'label':i['label']}
    news = add_feature(news)
    id += 1
    trained_dataset.append(news)

with open('True.csv', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for i in reader:
    news = {'id':id, 'headline':i['title'], 'content':i['text'], 'label':1}
    news = add_feature(news)
    id += 1
    trained_dataset.append(news)

# print(trained_dataset)

# train the model